In [1]:
import pandas as pd
import os
import sys
#import fastparquet
import numpy as np
#import tabix

from subprocess import Popen, PIPE

import gzip
from pybedtools import BedTool

In [2]:
#Input_sumstats=pd.read_parquet("/Users/xg1/Downloads/ldsc/input_sumstats/GCST005536.parquet")
#Input_sumstats.head()
from collections import OrderedDict

def Garfield_Prep_Pval(input_sumstat_path, out_path, to_expand_path):
    Input_sumstats=pd.read_parquet(input_sumstat_path, engine="pyarrow")
    for i in range(1, 23):
        Sumstats_bychr=Input_sumstats.loc[Input_sumstats['chrom'] == str(i)][["pos", "pval"]]
        Sumstats_bychr.to_csv(out_path+"chr"+str(i), sep="\t", index=False, header=None)

        To_expand=Input_sumstats.loc[Input_sumstats['chrom'] == str(i)][["chrom", "pos"]]
        To_expand['chrom']="chr"+To_expand['chrom']
        To_expand["SNP"]=To_expand.apply(lambda x: x['chrom']+"_"+str(x['pos']), axis=1)
        cols = OrderedDict([
        ('chrom', 'Chrom'),
        ('SNP', 'SNP'),
        ('pos', 'BP')
        ])
        To_expand=To_expand.loc[:, list(cols.keys())].rename(columns=cols)
        To_expand.to_csv(to_expand_path+"chr"+str(i), sep="\t", index=False)

#lo=LiftOver("hg38", "hg19")

def Garfield_Prep_Pval_and_LO(input_sumstat_path, out_path, to_expand_path):
    Input_sumstats=pd.read_parquet(input_sumstat_path, engine="pyarrow")
    for i in range(1, 23):
        Sumstats_bychr=Input_sumstats.loc[Input_sumstats['chrom'] == str(i)][["pos", "pval"]]
        Sumstats_bychr.to_csv(out_path+"chr"+str(i), sep="\t", index=False, header=None)


Garfield_Prep_Pval("/Users/xg1/Downloads/ldsc/input_sumstats/GCST005536.parquet", "/Users/xg1/Downloads/garfield-data/pval/T1D/", "/Users/xg1/Downloads/garfield-data/sumstats_bychr/T1D/")



In [3]:
import gzip
from pybedtools import BedTool

def Garfield_make_annot(input_sumstat_path, out_path, to_expand_path, bed_for_annot):
    Input_sumstats=pd.read_parquet(input_sumstat_path, engine="pyarrow")
    for i in range(1, 3):
        Sumstats_bychr=Input_sumstats.loc[Input_sumstats['chrom'] == str(i)][["pos", "pval"]]
    #    Sumstats_bychr.to_csv(out_path+"chr"+str(i), sep="\t", index=False, header=None)

        To_expand=Input_sumstats.loc[Input_sumstats['chrom'] == str(i)][["chrom", "pos"]]
        To_expand['chrom']="chr"+To_expand['chrom']
        To_expand["SNP"]=To_expand.apply(lambda x: x['chrom']+"_"+str(x['pos']), axis=1)
        cols = OrderedDict([
        ('chrom', 'Chrom'),
        ('SNP', 'SNP'),
        ('pos', 'BP')
        ])
        To_expand=To_expand.loc[:, list(cols.keys())].rename(columns=cols)
    #    To_expand.to_csv(to_expand_path+"chr"+str(i), sep="\t", index=False)

        To_overlap=To_expand[['Chrom', 'BP']]
        To_overlap['start']=To_overlap['BP']-1
        To_overlap=To_overlap[['Chrom', 'start', 'BP']]

        SNP_bed=BedTool.from_dataframe(To_overlap)
        Annot_bed=BedTool(bed_for_annot)

        annot_out=SNP_bed.intersect(Annot_bed)
        bp=[x.start + 1 for x in annot_out]
        df_int=pd.DataFrame({'BP': bp, 'ANNOT': 1})
        df_annot=pd.merge(To_expand, df_int, how="left", on='BP')
        df_annot.fillna(0, inplace=True)

        

        return(df_annot)

#/Users/xg1/Downloads/ldsc/Comparison_enhancers/BSS00079.bed



In [34]:
abc=Garfield_make_annot("/Users/xg1/Downloads/ldsc/input_sumstats/GCST005536.parquet", 
    "/Users/xg1/Downloads/garfield-data/pval/T1D/", "/Users/xg1/Downloads/garfield-data/sumstats_bychr/T1D/",
    "/Users/xg1/Downloads/ldsc/Comparison_enhancers/test_enhancer.bed")

print(abc)

In [4]:
#for file in os.listdir("/Users/xg1/Downloads/ldsc/Comparison_enhancers/"):
#    print(file)

def Garfield_make_annot(input_sumstat_path, out_path, to_expand_path, bed_for_annot_dir):
    Input_sumstats=pd.read_parquet(input_sumstat_path, engine="pyarrow")
    
    for i in range(21, 23):
        Sumstats_bychr=Input_sumstats.loc[Input_sumstats['chrom'] == str(i)][["pos", "pval"]]
    #    Sumstats_bychr.to_csv(out_path+"chr"+str(i), sep="\t", index=False, header=None)

        To_expand=Input_sumstats.loc[Input_sumstats['chrom'] == str(i)][["chrom", "pos"]]
        To_expand['chrom']="chr"+To_expand['chrom']
        To_expand["SNP"]=To_expand.apply(lambda x: x['chrom']+"_"+str(x['pos']), axis=1)
        cols = OrderedDict([
        ('chrom', 'Chrom'),
        ('SNP', 'SNP'),
        ('pos', 'BP')
        ])
        To_expand=To_expand.loc[:, list(cols.keys())].rename(columns=cols)
    #    To_expand.to_csv(to_expand_path+"chr"+str(i), sep="\t", index=False)

        To_overlap=To_expand[['Chrom', 'BP']]
        To_overlap['start']=To_overlap['BP']-1
        To_overlap=To_overlap[['Chrom', 'start', 'BP']]

        Annot_out=To_overlap['BP']

        SNP_bed=BedTool.from_dataframe(To_overlap)
        for bed_for_annot in os.listdir(bed_for_annot_dir):
            Annot_bed=BedTool(bed_for_annot_dir+bed_for_annot)

            SNP_intersect=SNP_bed.intersect(Annot_bed)
            bp=[x.start + 1 for x in SNP_intersect]
            df_int=pd.DataFrame({'BP': bp, bed_for_annot: int(1)})
            Annot_out=pd.merge(Annot_out, df_int, how="left", on='BP')
            Annot_out.fillna(0, inplace=True)

        Annot_out=Annot_out.astype(int)
        Annot_out['annotations']=Annot_out.iloc[:,1:].apply(lambda row: "".join(row.values.astype(str)), axis=1)
        Annot_out=Annot_out[['BP', 'annotations']]

        return(Annot_out)


In [5]:
import time
def Garfield_annot_UK10K(input_SNP_path, out_path, bed_for_annot_dir):
    for i in range(1, 23):
        start_time=time.time()

        Input_SNPs=pd.read_csv(input_SNP_path+"chr"+str(i), names=["BP", "MAF", "TSS"], sep=" ")
        Input_SNPs['Chrom']="chr"+str(i)

        To_overlap=Input_SNPs[['Chrom', 'BP']]
        To_overlap['start']=To_overlap['BP'].astype(int)-1
        To_overlap=To_overlap[['Chrom', 'start', 'BP']]

        Annot_out=To_overlap['BP']

        SNP_bed=BedTool.from_dataframe(To_overlap)
        for bed_for_annot in os.listdir(bed_for_annot_dir):
            Annot_bed=BedTool(bed_for_annot_dir+bed_for_annot)

            SNP_intersect=SNP_bed.intersect(Annot_bed)
            bp=[x.start + 1 for x in SNP_intersect]
            df_int=pd.DataFrame({'BP': bp, bed_for_annot: int(1)})
            Annot_out=pd.merge(Annot_out, df_int, how="left", on='BP')
            Annot_out.fillna(0, inplace=True)

        Annot_out=Annot_out.astype(int)
        Annot_out['annotations']=Annot_out.iloc[:,1:].apply(lambda row: "".join(row.values.astype(str)), axis=1)
        Annot_out=Annot_out[['BP', 'annotations']]
        Annot_out.to_csv(out_path+"chr"+str(i), sep=" ", header=None, index=False)
        print("finished chr"+str(i)+" in "+str(time.time()-start_time)+" seconds!")
        #return(Annot_out)

In [ ]:
Garfield_annot_UK10K("/Users/xg1/Downloads/garfield-data/maftssd/", 
    "/Users/xg1/Downloads/garfield-data/Comparison_annotations/",
    "/Users/xg1/Downloads/ldsc/Comparison_enhancers/")

Error: Session cannot generate requests

In [63]:
#df_new=abc.iloc[:,1:].apply(lambda row: "".join(row.values.round().astype(str)), axis=1)
#print(df_new)
abc.astype(int)

,BP,BSS00206.bed,BSS00207.bed,BSS00228.bed,BSS00189.bed,BSS00188.bed,BSS01068.bed,BSS01120.bed,BSS00511.bed,BSS01169.bed,...,BSS01071.bed,BSS01714.bed,BSS01113.bed,BSS01665.bed,BSS00234.bed,BSS01666.bed,BSS01712.bed,BSS00197.bed,test_enhancer.bed,BSS00196.bed
0,14022866,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,14144009,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,14145782,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,14173679,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,14176477,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3010,46380677,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3011,46534814,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3012,46550668,0,1,0,0,0,0,1,0,1,...,0,0,0,0,0,1,0,0,1,0
3013,46564886,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
Input_sumstats=pd.read_parquet("/Users/xg1/Downloads/ldsc/input_sumstats/GCST005536.parquet")
Input_sumstats.loc[Input_sumstats['chrom'] == str(1)]

,type,study_id,phenotype_id,bio_feature,gene_id,chrom,pos,ref,alt,beta,se,pval,n_total,n_cases,eaf,mac,mac_cases,info,is_cc
0,gwas,GCST005536,None,None,None,1,1182895,C,T,-0.073001,0.045001,0.104756,18856,6683,0.042950,1619.729175,574.069266,NaN,True
1,gwas,GCST005536,None,None,None,1,1199862,A,C,-0.038949,0.020973,0.063291,18856,6683,0.278139,10489.186727,3717.608979,NaN,True
2,gwas,GCST005536,None,None,None,1,1205055,G,T,-0.027988,0.019992,0.161513,18856,6683,0.329919,12441.909186,4409.698721,NaN,True
3,gwas,GCST005536,None,None,None,1,1228424,C,T,-0.050031,0.026017,0.054470,18856,6683,0.155267,5855.413264,2075.293108,NaN,True
4,gwas,GCST005536,None,None,None,1,1229930,G,A,-0.005013,0.039099,0.897987,18856,6683,0.055621,2097.581761,743.431211,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10802,gwas,GCST005536,None,None,None,1,248697340,A,C,0.024985,0.027984,0.371934,18856,6683,0.126461,4769.113016,1690.283320,NaN,True
10803,gwas,GCST005536,None,None,None,1,248847255,C,T,0.011039,0.042149,0.793395,18856,6683,0.051005,1923.483085,681.726636,NaN,True
10804,gwas,GCST005536,None,None,None,1,248908368,C,T,0.023033,0.063091,0.715052,18856,6683,0.021392,806.752236,285.931544,NaN,True
10805,gwas,GCST005536,None,None,None,1,248914941,A,G,-0.013015,0.059069,0.825607,18856,6683,0.976916,870.540267,308.539489,NaN,True


In [4]:
from subprocess import Popen, PIPE

def bgzip(filename):
    """Call bgzip to compress a file."""
    Popen(['bgzip', '-f', filename])

def tabix_index(filename,
        preset="gff", chrom=1, start=4, end=5, skip=0, comment="#"):
    """Call tabix to create an index for a bgzip-compressed file."""
    Popen(['tabix', '-p', preset, '-s', chrom, '-b', start, '-e', end,
        '-S', skip, '-c', comment])

def tabix_query(filename, chrom, start, end):
    """Call tabix and generate an array of strings for each line it returns."""
    query = '{}:{}-{}'.format(chrom, start, end)
    process = Popen(['tabix', '-f', filename, query], stdout=PIPE)
    for line in process.stdout:
        yield line.strip().split()

In [36]:
def snpLdTabix(snp, chrom, bp, tabixDir, window, r2min):
    """
    Retrive LD info from the tabix file for a single SNP.
    """
    file = os.path.join(tabixDir, '{}_GRCh38.EUR.ld.bgz')
    tabixFile = file.format(chrom)
    print(tabixFile)
    st = bp - window
    if st < 0:
        st = 0
    en = bp + window

    query = "tabix {tabixFile} {chrom}:{st}-{en} | awk '$6 >= {r2min} {{print $0}}' | grep -w {snp}"
    proc = Popen(query, shell=True, stdout=PIPE)

    return proc

In [37]:
abc=snpLdTabix("rs12260013", "chr10", 20364, "/User/xg1/Downloads/CHEERS/LD_GRCh38/", 500000, 0.1)

/User/xg1/Downloads/CHEERS/LD_GRCh38/chr10_GRCh38.EUR.ld.bgz


In [38]:
abc.stdout

<_io.BufferedReader name=61>

In [2]:
for line in abc.stdout:
    fields=line.strip().split()
    print(fields[0])

NameError: name 'abc' is not defined

In [ ]:
tabix_query("/User/xg1/Downloads/CHEERS/LD_GRCh38/chr10_GRCh38.EUR.ld.bgz", "chr10", 1000000, 1010000)

<generator object tabix_query at 0x7fc35b680c00>

AttributeError: 'generator' object has no attribute 'stdout'